# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 8:30AM,242489,10,8300961.1,"Citieffe, Inc.",Released
1,Jul 22 2022 8:30AM,242486,10,PRF-33782,Bio-PRF,Released
2,Jul 22 2022 8:30AM,242486,10,PRF-33785,Bio-PRF,Released
3,Jul 22 2022 8:30AM,242486,10,PRF-33791,Bio-PRF,Released
4,Jul 22 2022 8:30AM,242486,10,PRF-33794,Bio-PRF,Released
5,Jul 22 2022 6:30AM,242500,22,621632,"NBTY Global, Inc.",Released
6,Jul 22 2022 3:49AM,242499,10,8305283,"Citieffe, Inc.",Released
7,Jul 21 2022 5:55PM,242498,16,TASA-8305292DD,TASA USA Inc.,Released
8,Jul 21 2022 5:26PM,242497,16,8304723,Sartorius Bioprocess Solutions,Released
9,Jul 21 2022 5:26PM,242497,16,8304732,Sartorius Bioprocess Solutions,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
28,242494,Released,3
29,242497,Released,2
30,242498,Released,1
31,242499,Released,1
32,242500,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242494,NaN,NaN,3.0
242497,NaN,NaN,2.0
242498,NaN,NaN,1.0
242499,NaN,NaN,1.0
242500,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,47.0,6.0,23.0
242399,0.0,1.0,0.0
242408,0.0,0.0,1.0
242410,0.0,0.0,1.0
242411,0.0,0.0,4.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242348,47,6,23
242399,0,1,0
242408,0,0,1
242410,0,0,1
242411,0,0,4


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,47,6,23
1,242399,0,1,0
2,242408,0,0,1
3,242410,0,0,1
4,242411,0,0,4


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242348,47,6,23
1,242399,,1,
2,242408,,,1
3,242410,,,1
4,242411,,,4


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 8:30AM,242489,10,"Citieffe, Inc."
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF
5,Jul 22 2022 6:30AM,242500,22,"NBTY Global, Inc."
6,Jul 22 2022 3:49AM,242499,10,"Citieffe, Inc."
7,Jul 21 2022 5:55PM,242498,16,TASA USA Inc.
8,Jul 21 2022 5:26PM,242497,16,Sartorius Bioprocess Solutions
10,Jul 21 2022 4:40PM,242494,19,ACG North America LLC
13,Jul 21 2022 4:32PM,242493,19,"AdvaGen Pharma, Ltd"
14,Jul 21 2022 4:29PM,242492,19,"AdvaGen Pharma, Ltd"
18,Jul 21 2022 4:17PM,242490,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 22 2022 8:30AM,242489,10,"Citieffe, Inc.",,,1
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,,,4
2,Jul 22 2022 6:30AM,242500,22,"NBTY Global, Inc.",,,1
3,Jul 22 2022 3:49AM,242499,10,"Citieffe, Inc.",,,1
4,Jul 21 2022 5:55PM,242498,16,TASA USA Inc.,,,1
5,Jul 21 2022 5:26PM,242497,16,Sartorius Bioprocess Solutions,,,2
6,Jul 21 2022 4:40PM,242494,19,ACG North America LLC,,,3
7,Jul 21 2022 4:32PM,242493,19,"AdvaGen Pharma, Ltd",,,1
8,Jul 21 2022 4:29PM,242492,19,"AdvaGen Pharma, Ltd",,,4
9,Jul 21 2022 4:17PM,242490,19,ACG North America LLC,,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242489,10,"Citieffe, Inc.",1,,
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4,,
2,Jul 22 2022 6:30AM,242500,22,"NBTY Global, Inc.",1,,
3,Jul 22 2022 3:49AM,242499,10,"Citieffe, Inc.",1,,
4,Jul 21 2022 5:55PM,242498,16,TASA USA Inc.,1,,
5,Jul 21 2022 5:26PM,242497,16,Sartorius Bioprocess Solutions,2,,
6,Jul 21 2022 4:40PM,242494,19,ACG North America LLC,3,,
7,Jul 21 2022 4:32PM,242493,19,"AdvaGen Pharma, Ltd",1,,
8,Jul 21 2022 4:29PM,242492,19,"AdvaGen Pharma, Ltd",4,,
9,Jul 21 2022 4:17PM,242490,19,ACG North America LLC,3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242489,10,"Citieffe, Inc.",1,,
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4,,
2,Jul 22 2022 6:30AM,242500,22,"NBTY Global, Inc.",1,,
3,Jul 22 2022 3:49AM,242499,10,"Citieffe, Inc.",1,,
4,Jul 21 2022 5:55PM,242498,16,TASA USA Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242489,10,"Citieffe, Inc.",1.0,NaN,NaN
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4.0,NaN,NaN
2,Jul 22 2022 6:30AM,242500,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jul 22 2022 3:49AM,242499,10,"Citieffe, Inc.",1.0,NaN,NaN
4,Jul 21 2022 5:55PM,242498,16,TASA USA Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 22 2022 8:30AM,242489,10,"Citieffe, Inc.",1.0,0.0,0.0
1,Jul 22 2022 8:30AM,242486,10,Bio-PRF,4.0,0.0,0.0
2,Jul 22 2022 6:30AM,242500,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jul 22 2022 3:49AM,242499,10,"Citieffe, Inc.",1.0,0.0,0.0
4,Jul 21 2022 5:55PM,242498,16,TASA USA Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2182275,76.0,2.0,7.0
12,242445,0.0,1.0,0.0
16,1697092,11.0,1.0,0.0
19,1212395,12.0,0.0,0.0
20,727237,42.0,6.0,47.0
21,727322,2.0,1.0,0.0
22,242500,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2182275,76.0,2.0,7.0
1,12,242445,0.0,1.0,0.0
2,16,1697092,11.0,1.0,0.0
3,19,1212395,12.0,0.0,0.0
4,20,727237,42.0,6.0,47.0
5,21,727322,2.0,1.0,0.0
6,22,242500,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,2.0,7.0
1,12,0.0,1.0,0.0
2,16,11.0,1.0,0.0
3,19,12.0,0.0,0.0
4,20,42.0,6.0,47.0
5,21,2.0,1.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,12,Released,0.0
2,16,Released,11.0
3,19,Released,12.0
4,20,Released,42.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21,22
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,47.0,0.0,0.0
Executing,2.0,1.0,1.0,0.0,6.0,1.0,0.0
Released,76.0,0.0,11.0,12.0,42.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21,22
0,Completed,7.0,0.0,0.0,0.0,47.0,0.0,0.0
1,Executing,2.0,1.0,1.0,0.0,6.0,1.0,0.0
2,Released,76.0,0.0,11.0,12.0,42.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21,22
0,Completed,7.0,0.0,0.0,0.0,47.0,0.0,0.0
1,Executing,2.0,1.0,1.0,0.0,6.0,1.0,0.0
2,Released,76.0,0.0,11.0,12.0,42.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()